## Step one download the file 
### change your url 


In [16]:
# download the data from SeaWiFS: http://orca.science.oregonstate.edu/1080.by.2160.monthly.hdf.vgpm.s.chl.a.sst.php

import requests
import os

def download_file(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

# make sure 'data/' directory exists
if not os.path.exists('data/'):
    os.makedirs('data/')

#############################################################################################################
##your code#################################################################################################
#############################################################################################################
base_url = 'http://orca.science.oregonstate.edu/data/1x2/monthly/mld030.hycom/hdf/mld.hycom_030.{}.tar'

for year in range(1997, 2024):  # 2010 is not included
    url = base_url.format(year)
    filename = f'data/mld.hycom_030.{year}.tar'
    download_file(url, filename)


# download the data from MODIS:  http://orca.science.oregonstate.edu/1080.by.2160.monthly.hdf.vgpm.m.chl.m.sst.php

#############################################################################################################
##your code#################################################################################################
#############################################################################################################
# new_url = 'http://orca.science.oregonstate.edu/data/1x2/monthly/vgpm.r2022.m.chl.m.sst/hdf/vgpm.m.{}.tar'

# for year in range(2010, 2024):  # 2024 is not included
#     url = new_url.format(year)
#     filename = f'data/vgpm.s.{year}.tar'
#     download_file(url, filename)

## Step two calculate the mean of whole data
### please check your data and the final dataset , name it properly 

In [1]:
import os
import tarfile
import gzip
import tempfile
from pyhdf.SD import SD, SDC
import pandas as pd
import numpy as np

# Define the latitude and longitude arrays
lats = np.linspace(90, -90, 1080)
lons = np.linspace(-180, 180, 2160)

# Create a meshgrid of latitudes and longitudes
lon_grid, lat_grid = np.meshgrid(lons, lats, indexing='ij')

# Flatten the arrays for dataframe construction
lat_flat = lat_grid.flatten()
lon_flat = lon_grid.flatten()

# Create a multi-index from the latitude and longitude arrays
index = pd.MultiIndex.from_arrays([lat_flat, lon_flat], names=['Lat', 'Long'])

# Create an empty DataFrame with this index
df = pd.DataFrame(index=index)

# Loop through each tar file
for year in range(1997, 2024):
    # Open the tar file
    tar_filename = f'data/mld.hycom_030.{year}.tar'
    with tarfile.open(tar_filename, "r:") as tar:
        
        # Create an empty DataFrame to store data for this year
        temp_df = pd.DataFrame(index=index)

        # Loop through each member of the tar file
        for member in tar.getmembers():
            # If it's a .hdf.gz file
            if member.name.endswith('.hdf.gz'):
                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
###your code#################################################################################################
### change 'mld' to '' ##################################################################################
                        # Access the 'mld' dataset
                        data = hdf_file.select('mld')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Flatten the data and add it to the temporary DataFrame
                        data_flat = data.flatten()
                        temp_df[member.name] = data_flat

                        # Close the file
                        hdf_file.end()

        # After going through all files for the year, calculate the mean for each location (ignoring NaNs)
        mean_data = temp_df.mean(axis=1, skipna=True)

        # Add this DataFrame to the main DataFrame
        df[str(year)] = mean_data

# Reset the index of the DataFrame, making 'Lat' and 'Long' normal columns
df.reset_index(inplace=True)

# Save the DataFrame to a CSV file
df.to_csv('mld_means.csv', index=False)





In [2]:
df_mld_means = pd.read_csv('mld_means.csv')
df_mld_means.head()

,Lat,Long,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,90.000000,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.826600,27.211823,32.903030,32.963802,32.608980,46.673367
1,89.833179,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.837618,27.215448,32.909860,32.967907,32.610250,46.677254
2,89.666358,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.868065,27.213402,32.911903,32.970306,32.611500,46.679893
3,89.499537,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.875278,27.214859,32.917380,32.973010,32.611800,46.676773
4,89.332715,-180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30.876595,27.214561,32.918095,32.979294,32.613052,46.680485


In [23]:
df_mld_means.describe()

,Lat,Long,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
count,2.332800e+06,2.332800e+06,1.408276e+06,1.408276e+06,1.408276e+06,1.408276e+06,1.408276e+06,1.408276e+06,1.408276e+06,1.408276e+06,...,1.402017e+06,1.402017e+06,1.402017e+06,1.402017e+06,1.535387e+06,1.533697e+06,1.533697e+06,1.533697e+06,1.533697e+06,1.533697e+06
mean,-6.736998e-15,-2.410972e-15,6.149137e+01,6.294367e+01,6.257512e+01,6.286864e+01,6.066013e+01,6.026777e+01,5.962403e+01,5.992304e+01,...,6.004174e+01,6.228459e+01,6.103743e+01,6.081088e+01,5.912681e+01,4.854502e+01,4.774283e+01,4.881339e+01,4.879551e+01,4.675138e+01
std,5.200967e+01,1.039712e+02,4.097085e+01,3.778995e+01,3.794584e+01,3.849731e+01,3.607759e+01,3.540604e+01,3.565189e+01,3.603428e+01,...,3.744433e+01,4.031214e+01,3.817451e+01,3.758844e+01,3.664786e+01,2.655985e+01,2.688019e+01,2.850043e+01,2.807238e+01,3.471590e+01
min,-9.000000e+01,-1.800000e+02,4.000000e+00,3.993215e+00,3.986439e+00,3.990579e+00,3.995166e+00,3.999180e+00,3.988292e+00,3.998876e+00,...,9.497684e+00,9.514389e+00,9.511917e+00,9.517311e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
25%,-4.500000e+01,-9.000000e+01,3.582031e+01,3.866857e+01,3.813681e+01,3.819613e+01,3.740963e+01,3.743136e+01,3.689331e+01,3.671695e+01,...,3.368140e+01,3.425556e+01,3.357554e+01,3.404565e+01,3.408250e+01,3.122709e+01,3.079055e+01,3.104953e+01,3.116683e+01,2.690473e+01
50%,-7.105427e-15,-1.425249e-14,4.695459e+01,5.060216e+01,5.073407e+01,5.067403e+01,4.986629e+01,4.927116e+01,4.845621e+01,4.885675e+01,...,4.810646e+01,4.790160e+01,4.807678e+01,4.841512e+01,4.712770e+01,4.159988e+01,4.084117e+01,4.187149e+01,4.136847e+01,3.966084e+01
75%,4.500000e+01,9.000000e+01,7.562315e+01,7.932338e+01,7.681049e+01,7.733212e+01,7.456791e+01,7.482238e+01,7.357315e+01,7.408436e+01,...,7.698699e+01,8.040919e+01,8.108750e+01,7.960258e+01,7.565919e+01,5.658365e+01,5.530078e+01,5.597609e+01,5.760262e+01,5.625160e+01
max,9.000000e+01,1.800000e+02,3.771343e+02,3.795920e+02,3.788337e+02,3.790154e+02,3.768887e+02,3.798910e+02,3.769129e+02,3.800199e+02,...,3.858532e+02,3.848569e+02,3.848225e+02,3.849182e+02,3.827087e+02,3.919591e+02,3.307609e+02,2.284897e+02,2.377766e+02,3.925975e+02


## Step three read cleaned excel

> read cleadned excel file

In [13]:
# read new data
# read each page from excel file
import pandas as pd
import numpy as np

filename='cleaned'
Pacific=pd.read_excel(filename+'.xlsx',sheet_name='Pacific')
Atlantic=pd.read_excel(filename+'.xlsx',sheet_name='Atlantic')
Mediterranean=pd.read_excel(filename+'.xlsx',sheet_name='Mediterranean')
Southern=pd.read_excel(filename+'.xlsx',sheet_name='Southern Ocean')
Arctic=pd.read_excel(filename+'.xlsx',sheet_name='Arctic')

## Step four melt for merge

> melt for merge 

In [16]:
# melt the data
#############################################################################################################
##your code#################################################################################################
# df_npp_melt = df_npp_means_new.melt(id_vars=['Lat', 'Long'], var_name='Year', value_name='npp')
# df_chl_melt = df_chl_means_new.melt(id_vars=['Lat', 'Long'], var_name='Year', value_name='chl')
# df_par_melt = df_par_means_new.melt(id_vars=['Lat', 'Long'], var_name='Year', value_name='par')
# df_sst_melt = df_sst_means_new.melt(id_vars=['Lat', 'Long'], var_name='Year', value_name='sst')
df_mld_melt = df_mld_means.melt(id_vars=['Lat', 'Long'], var_name='Year', value_name='mld')

## Step five Select decimal 

In [17]:
# Then merge on these columns
# assuming df_melted_variable1-5 and ocean_dfs are your dataframes
ocean_names = ["Pacific", "Atlantic", "Mediterranean", "Southern", "Arctic"]
# df_melted_list = [df_npp_melt, df_chl_melt, df_par_melt, df_sst_melt, df_mld_melt]
#############################################################################################################
##your code#################################################################################################
#############################################################################################################

# change this to your dataframes
df_melted_list = [df_mld_melt]
ocean_dfs = [Pacific, Atlantic, Mediterranean, Southern, Arctic]


for df in df_melted_list + ocean_dfs:
    # convert types safely
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')
    df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
    df['Long'] = pd.to_numeric(df['Long'], errors='coerce')


# round lat and long to 0.0001
# I believe round is a way of thresholding the data, let me know if there is a better wayof matching 
# the lat and longs
for df in df_melted_list + ocean_dfs:
    df['Lat'] = df['Lat'].round(1)
    df['Long'] = df['Long'].round(1)



## check if there is a match 

In [ ]:
# Create a set of tuples for the Pacific dataframe
pacific_coords = set(zip(Pacific['Lat'], Pacific['Long']))

# Create a set of tuples for the df_npp dataframe
npp_coords = set(zip(df_mld_melt['Lat'], df_mld_melt['Long']))

# Find the common coordinates
common_coords = pacific_coords.intersection(npp_coords)

print(f"Common coordinates between Pacific and df_npp: {len(common_coords)}")


## Step six merge 

In [19]:
import pandas as pd
import numpy as np

merged_dfs = []  # List to store merged dataframes

for ocean_name, ocean_df in zip(ocean_names, ocean_dfs):
    # merge ocean_df with all dataframes in df_melted_list
    for df_melted in df_melted_list:
        ocean_df = pd.merge(ocean_df, df_melted, on=['Lat', 'Long', 'Year'], how='left')

    # Fill NaN values with the mean of each column
    for col in ocean_df.select_dtypes(include=[np.number]).columns:
        ocean_df[col] = ocean_df[col].astype(float)
        ocean_df[col].fillna(ocean_df[col].mean(), inplace=True)
        if ocean_df[col].apply(float.is_integer).all():  # Check if all values are integer
            ocean_df[col] = ocean_df[col].astype('Int64')  # Change dtype back to integer

    merged_dfs.append(ocean_df)  # Append the merged dataframe to the list

with pd.ExcelWriter('merged.xlsx') as writer:
    for ocean_name, merged_df in zip(ocean_names, merged_dfs):
        merged_df.to_excel(writer, sheet_name=ocean_name)  # Write each merged dataframe to a different sheet
